In [2]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from pathlib import Path
import faiss

c:\Users\Kistanov\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chunk_files = sorted(Path('.').glob('preproc_data*.parquet'))
dfs = [pd.read_parquet(f) for f in chunk_files]
df = pd.concat(dfs, ignore_index=True)

In [4]:
texts = df['lemmatized_text'].tolist() 


In [5]:
model = SentenceTransformer('cointegrated/rubert-tiny2')

embeddings = model.encode(texts, show_progress_bar=True, batch_size=64) # create embeddings
embeddings = np.array(embeddings)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True) # normalize it


Batches: 100%|██████████| 777/777 [46:06<00:00,  3.56s/it]  


In [6]:
import numpy as np

index = faiss.IndexFlatIP(embeddings.shape[1])  # create index
index.add(embeddings)  # add embeddings to index
np.save("embeddings.npy", embeddings)  # save embeddings to file    